### Imports
Import Python modules for executing the notebook.  
 - Pandas is used for building and handling dataframes.  
 - Scrapbook is used for recording data for the Notebook Execution Service. 
 - systems_api is an NI provided package for communicating with the SystemLink Systems service
 - query_systems_request is an NI provided package for building queries to the SystemLink Systems service

In [ ]:
import pandas as pd
import scrapbook as sb

from systemlink.clients.nisysmgmt.api.systems_api import SystemsApi
from systemlink.clients.nisysmgmt.models.query_systems_request import QuerySystemsRequest

### Parameters
 - `group_by`: The property by which data is grouped. This is unused in this simple example, but for the data to be available as a column in the Systems Grid, we must at minimum support 'System' here. Other data groupings could be used for multi-purpose notebooks.  
 - `package`: The Package Name of the software you would like to display the version of in the Systems Grid.
 - `systems_filter`: A filter, as provided to the http API, that specifies which systems to validate against the input state. An empty filter will match all systems.

Parameters are also listed in the metadata for the parameters cell, along with their default values.
To see the metadata, select the code cell below and click the wrench icon in the far right panel.

In [ ]:
group_by = "System"
package = ""
systems_filter = ""

### Query for Systems with the specified package and get the Package Version on each system

In [ ]:
api = SystemsApi()

projection = f"new(id, packages.data[\"{package}\"].displayversion, packages.data[\"{package}\"].version)"
filter = (systems_filter or "!string.IsNullOrEmpty(id)") + f" && packages.data.keys.Contains(\"{package}\")"

query_sys_request = QuerySystemsRequest(skip=0, projection=projection, filter=filter)
query_result = api.get_systems_by_query(query=query_sys_request)
data = await query_result

### Extract Package data from query results and create pandas dataframe

In [ ]:
pkg_version = { item['id'] : item['displayversion'] for item in data.data }
df = pd.DataFrame.from_dict(pkg_version, orient='index', columns=['Package Version'])
df

### Convert dataframe to result format that the Systems Grid can interpret

In [ ]:
df_dict = {
    'columns': ['minion id', 'package version'],
    'values': df.reset_index().values.tolist()
}

result = [{
    "display_name": "Package Version",
    "id": 'package_version',
    "type": 'data_frame',
    "data": df_dict
}]

sb.glue('result', result)

### View the output of this report in the Systems Grid (SystemLink 2021 R2 and later)
1. On your SystemLink Server, ensure the NI SystemLink Server - JupyterHub Module is installed
1. In SystemLink, navigate to Utilities -> Jupyter, and create a folder named 'reports' if it does not already exist
1. Upload this notebook to the reports folder
1. From the Systems page, press the edit grid button in the upper-right section of the grid area
1. Press the '+ ADD' button to add a new column, and select 'Notebook' as the data source
1. Select this report in the Path field ([your_username]/reports/PackageVersion.ipynb)
1. In the Output field, select 'Package Version'
1. In the Package field, enter the name of the package you'd like to display the version of (eg. ni-daqmx, ni-teststand-2019)
   1. This can be found in the header of the hover text of the package in NIPM
   1. It can also be found in the Installed Software tab of a System in SystemLink by showing the Package Name column
1. Select an appropriate Update interval for your needs (5 min to 24 hours)
1. Enter an appropriate Column name (ex. 'DAQmx Version') and press Done